ASF Search and SBAS Stack Initialization

In [3]:
from datetime import date
import asf_search as asf
from pprint import pprint

# Define date range around the Blatten landslide event (28 May 2025)
stack_start = date(2025, 4, 15)
stack_end = date(2025, 6, 20)

# Define ASF search parameters
opts = asf.ASFSearchOptions(**{
    "maxResults": 5000,
    "bbox": [7.6172, 46.2195, 8.0172, 46.6195],  # Blatten AOI (Switzerland)
    "beamSwath": ["IW"],
    "flightDirection": "DESCENDING",  # Use DESCENDING first, see note below
    "polarization": ["VV+VH", "VV"],
    "processingLevel": ["SLC"],
    "start": stack_start.isoformat(),
    "end": stack_end.isoformat(),
    "dataset": ["SENTINEL-1"]
})

# Search SLC scenes from ASF
search_results = asf.search(opts=opts)
print(f"Found {len(search_results)} scenes.")

# Create baseline stack (SBAS-compatible) from the most recent scene
baseline_results = asf.baseline_search.stack_from_product(search_results[-1])
print(f"Generated {len(baseline_results)} SBAS pairs.")

# Preview first few SBAS pairs
pprint(baseline_results[:3])


/etc/timezone is deprecated on Debian, and no longer reliable. Ignoring.
/mnt/data/tmp/ipykernel_1124523/1628721161.py:10: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
  opts = asf.ASFSearchOptions(**{
/mnt/data/tmp/ipykernel_1124523/1628721161.py:23: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
  search_results = asf.search(opts=opts)


Found 24 scenes.
Generated 478 SBAS pairs.


Make some directories 

In [28]:
import os
import subprocess
import glob

# === 1. Define GMTSAR-standard paths ===
work_dir = "/mnt/data/gmtsar_test_1"
download_dir = os.path.join(work_dir, "raw")
dem_dir = os.path.join(work_dir, "topo")
orbit_dir = download_dir  # Orbit files go under raw

os.makedirs(download_dir, exist_ok=True)
os.makedirs(dem_dir, exist_ok=True)

print(f"DEM directory: {dem_dir}")
print(f"RAW SLC directory: {download_dir}")
print(f"Orbit directory: {orbit_dir}")





DEM directory: /mnt/data/gmtsar_test_1/topo
RAW SLC directory: /mnt/data/gmtsar_test_1/raw
Orbit directory: /mnt/data/gmtsar_test_1/raw


Make a dataframe of search results 

In [30]:
import pandas as pd

# Convert to DataFrame
search_df = pd.DataFrame([result.properties for result in search_results])

# Check available columns
print("Available columns:\n", search_df.columns.tolist())

# Preview first few rows
search_df[['sceneName', 'platform', 'polarization','beamModeType','orbit', 'startTime']].head()


Available columns:
 ['centerLat', 'centerLon', 'stopTime', 'fileID', 'flightDirection', 'pathNumber', 'processingLevel', 'url', 'startTime', 'sceneName', 'browse', 'platform', 'bytes', 'md5sum', 'frameNumber', 'granuleType', 'orbit', 'polarization', 'processingDate', 'sensor', 'groupID', 'pgeVersion', 'fileName', 'beamModeType', 's3Urls']


,sceneName,platform,polarization,beamModeType,orbit,startTime
0,S1A_IW_SLC__1SDV_20250619T054333_20250619T0544...,Sentinel-1A,VV+VH,IW,59711,2025-06-19T05:43:33Z
1,S1A_IW_SLC__1SDV_20250614T053536_20250614T0536...,Sentinel-1A,VV+VH,IW,59638,2025-06-14T05:35:36Z
2,S1C_IW_SLC__1SDV_20250613T054227_20250613T0542...,Sentinel-1C,VV+VH,IW,2760,2025-06-13T05:42:27Z
3,S1C_IW_SLC__1SDV_20250608T053437_20250608T0535...,Sentinel-1C,VV+VH,IW,2687,2025-06-08T05:34:37Z
4,S1C_IW_SLC__1SDV_20250608T053413_20250608T0534...,Sentinel-1C,VV+VH,IW,2687,2025-06-08T05:34:13Z


In [31]:
search_df.to_csv(os.path.join(work_dir, "search_df.csv"), index=False)


In [47]:
print(f"Search returned {len(search_df)} scenes.")
print("Date range:", search_df['startTime'].min(), "to", search_df['startTime'].max())


Search returned 24 scenes.
Date range: 2025-04-15T05:35:37Z to 2025-06-19T05:43:33Z


Extract a stack product in my time span 

In [32]:
from dateutil.parser import parse as parse_date
import pandas as pd
# Extract properties and geometry from baseline results
list(scene.properties.values() for scene in baseline_results[:3])
# Create a DataFrame from the baseline results
stack = pd.DataFrame([
    list(scene.properties.values()) + [scene.geometry]
    for scene in baseline_results
    if stack_start <= parse_date(scene.properties['startTime']) <= stack_end
], columns=list(baseline_results[0].properties.keys()) + ['geometry'])
# Show available columns
print("Available columns:\n", stack.columns.tolist())

# Preview the first few rows
stack[['sceneName', 'startTime', 'stopTime']].head()

Available columns:
 ['centerLat', 'centerLon', 'stopTime', 'fileID', 'flightDirection', 'pathNumber', 'processingLevel', 'url', 'startTime', 'sceneName', 'browse', 'platform', 'bytes', 'md5sum', 'frameNumber', 'granuleType', 'orbit', 'polarization', 'processingDate', 'sensor', 'groupID', 'pgeVersion', 'fileName', 'beamModeType', 's3Urls', 'temporalBaseline', 'perpendicularBaseline', 'geometry']


,sceneName,startTime,stopTime
0,S1A_IW_SLC__1SDV_20250415T053537_20250415T0536...,2025-04-15T05:35:37Z,2025-04-15T05:36:04Z
1,S1A_IW_SLC__1SDV_20250427T053538_20250427T0536...,2025-04-27T05:35:38Z,2025-04-27T05:36:05Z
2,S1C_IW_SLC__1SDV_20250503T053434_20250503T0535...,2025-05-03T05:34:34Z,2025-05-03T05:35:01Z
3,S1A_IW_SLC__1SDV_20250509T053537_20250509T0536...,2025-05-09T05:35:37Z,2025-05-09T05:36:04Z
4,S1C_IW_SLC__1SDV_20250515T053435_20250515T0535...,2025-05-15T05:34:35Z,2025-05-15T05:35:02Z


In [104]:
# Check available columns
print("Available columns:\n", stack.columns.tolist())

# Loop through and print geometry WKT or coordinates
for i, geom in enumerate(stack['geometry']):
    print(f"--- Geometry {i+1} ---")
    print(geom)


Available columns:
 ['centerLat', 'centerLon', 'stopTime', 'fileID', 'flightDirection', 'pathNumber', 'processingLevel', 'url', 'startTime', 'sceneName', 'browse', 'platform', 'bytes', 'md5sum', 'frameNumber', 'granuleType', 'orbit', 'polarization', 'processingDate', 'sensor', 'groupID', 'pgeVersion', 'fileName', 'beamModeType', 's3Urls', 'temporalBaseline', 'perpendicularBaseline', 'geometry']
--- Geometry 1 ---
{'coordinates': [[[10.252147, 46.595284], [6.912548, 46.993725], [6.553195, 45.376583], [9.79403, 44.978661], [10.252147, 46.595284]]], 'type': 'Polygon'}
--- Geometry 2 ---
{'coordinates': [[[10.248555, 46.595936], [6.908627, 46.99435], [6.549314, 45.377087], [9.790461, 44.979187], [10.248555, 46.595936]]], 'type': 'Polygon'}
--- Geometry 3 ---
{'coordinates': [[[10.086966, 45.932964], [6.773118, 46.333443], [6.416841, 44.716366], [9.634801, 44.316227], [10.086966, 45.932964]]], 'type': 'Polygon'}
--- Geometry 4 ---
{'coordinates': [[[10.249989, 46.59584], [6.910178, 46.99422

In [106]:
# Initialize overall bounds
min_lon, min_lat = float('inf'), float('inf')
max_lon, max_lat = float('-inf'), float('-inf')

# Loop through geometries and extract bounds
for i, geom in enumerate(stack['geometry']):
    coords = geom['coordinates'][0]  # Assuming Polygon, not MultiPolygon
    print(f"--- Geometry {i+1} ---")
    for lon, lat in coords:
        print(f"Lon: {lon}, Lat: {lat}")
        # Update global bounds
        min_lon = min(min_lon, lon)
        max_lon = max(max_lon, lon)
        min_lat = min(min_lat, lat)
        max_lat = max(max_lat, lat)

# Print overall WESN bounds
print("\n[Overall Bounding Box for All Geometries]")
print(f"W: {min_lon}, E: {max_lon}, S: {min_lat}, N: {max_lat}")


--- Geometry 1 ---
Lon: 10.252147, Lat: 46.595284
Lon: 6.912548, Lat: 46.993725
Lon: 6.553195, Lat: 45.376583
Lon: 9.79403, Lat: 44.978661
Lon: 10.252147, Lat: 46.595284
--- Geometry 2 ---
Lon: 10.248555, Lat: 46.595936
Lon: 6.908627, Lat: 46.99435
Lon: 6.549314, Lat: 45.377087
Lon: 9.790461, Lat: 44.979187
Lon: 10.248555, Lat: 46.595936
--- Geometry 3 ---
Lon: 10.086966, Lat: 45.932964
Lon: 6.773118, Lat: 46.333443
Lon: 6.416841, Lat: 44.716366
Lon: 9.634801, Lat: 44.316227
Lon: 10.086966, Lat: 45.932964
--- Geometry 4 ---
Lon: 10.249989, Lat: 46.59584
Lon: 6.910178, Lat: 46.994221
Lon: 6.550885, Lat: 45.376953
Lon: 9.791917, Lat: 44.979088
Lon: 10.249989, Lat: 46.59584
--- Geometry 5 ---
Lon: 10.086464, Lat: 45.933083
Lon: 6.772568, Lat: 46.333534
Lon: 6.416298, Lat: 44.716331
Lon: 9.634297, Lat: 44.316219
Lon: 10.086464, Lat: 45.933083
--- Geometry 6 ---
Lon: 10.251045, Lat: 46.595608
Lon: 6.911162, Lat: 46.994015
Lon: 6.551862, Lat: 45.376755
Lon: 9.792971, Lat: 44.978863
Lon: 10.2

Save a pkl of the serach object in the current pwd and in the work_dir

In [33]:
import pickle

# Save the object
with open("search_results.pkl", "wb") as f:
    pickle.dump(search_results, f)
with open(os.path.join(work_dir, "search_results.pkl"), "wb") as f:
    pickle.dump(search_results, f)
# Load it later
with open("search_results.pkl", "rb") as f:
    search_results = pickle.load(f)


Save a csv of the stack 

In [112]:
stack.to_csv(os.path.join(work_dir, "sbas_stack_metadata.csv"), index=False)


Printout the bbox based on teh geometries from stack

In [35]:
from shapely.geometry import shape
import numpy as np
import pandas as pd

# Convert geometries to shapely objects
geometries = [shape(g) for g in stack["geometry"]]
all_coords = np.vstack([np.array(geom.exterior.coords) for geom in geometries])

# Compute bounding box
min_lon, min_lat = map(float, np.min(all_coords, axis=0))
max_lon, max_lat = map(float, np.max(all_coords, axis=0))

bbox = [round(min_lon, 4), round(min_lat, 4), round(max_lon, 4), round(max_lat, 4)]

# Date range
start_dates = pd.to_datetime(stack["startTime"])
min_date = start_dates.min().date()
max_date = start_dates.max().date()

# Output
print("Clean Bounding Box:", bbox)
print("Date Range:", min_date, "to", max_date)



Clean Bounding Box: [6.4116, 44.3162, 10.2521, 46.9944]
Date Range: 2025-04-15 to 2025-06-14


Create all possible interferometric pairs

In [36]:
from itertools import combinations

# Create all possible unique interferometric pairs (master, slave)
pairs = list(combinations(stack.sort_values('startTime')['sceneName'], 2))

# Or as DataFrame for clarity
pairs_df = pd.DataFrame(pairs, columns=['master', 'slave'])

print("Total pairs:", len(pairs_df))
pairs_df.head()


Total pairs: 45


,master,slave
0,S1A_IW_SLC__1SDV_20250415T053537_20250415T0536...,S1A_IW_SLC__1SDV_20250427T053538_20250427T0536...
1,S1A_IW_SLC__1SDV_20250415T053537_20250415T0536...,S1C_IW_SLC__1SDV_20250503T053434_20250503T0535...
2,S1A_IW_SLC__1SDV_20250415T053537_20250415T0536...,S1A_IW_SLC__1SDV_20250509T053537_20250509T0536...
3,S1A_IW_SLC__1SDV_20250415T053537_20250415T0536...,S1C_IW_SLC__1SDV_20250515T053435_20250515T0535...
4,S1A_IW_SLC__1SDV_20250415T053537_20250415T0536...,S1A_IW_SLC__1SDV_20250521T053537_20250521T0536...


Define time range and InSAR processing parameters 

In [37]:
from dateutil.parser import parse as parse_date

# Your known date range
start_date = '2025-04-15'
end_date = '2025-06-19'

# Parse to datetime and extract year & seasonal window
stack_start = parse_date(start_date + ' 00:00:00Z')
stack_end = parse_date(end_date + ' 00:00:00Z')

year_start = stack_start.year
year_end = stack_end.year
season_period = [start_date[5:], end_date[5:]]

# Processing parameters
max_temporal_baseline = 36              # Maximum days between pairs
interannual_baseline_max = 380          # Not applicable here, but defined
interannual_baseline_min = 350
look_nums = '20x4'                      # Range x Azimuth multilook factors
phase_filter_para = 0.8                 # Exponent for adaptive phase filtering


Filter stack for seasonal subset by year


In [39]:
from dateutil.parser import parse as parse_date

stack['startTime'] = stack['startTime'].apply(parse_date)
stack['startTime'] = stack['startTime'].dt.tz_localize(None)  # remove timezone if needed


stack_season = pd.DataFrame()
for year_index in range(int(year_start), int(year_end) + 1):
    season_start = parse_date(f'{str(year_index)}-{season_period[0]} 00:00:00Z').replace(tzinfo=None)
    season_end = parse_date(f'{str(year_index)}-{season_period[1]} 00:00:00Z').replace(tzinfo=None)
    stack_season = pd.concat([
        stack_season,
        stack.loc[(season_start <= stack.startTime) & (stack.startTime <= season_end)]
    ])


In [41]:
print("Available columns:\n", stack_season.columns.tolist())

# Preview the first few rows
stack_season[['sceneName', 'startTime', 'stopTime','temporalBaseline', 'perpendicularBaseline','geometry']].head()

Available columns:
 ['centerLat', 'centerLon', 'stopTime', 'fileID', 'flightDirection', 'pathNumber', 'processingLevel', 'url', 'startTime', 'sceneName', 'browse', 'platform', 'bytes', 'md5sum', 'frameNumber', 'granuleType', 'orbit', 'polarization', 'processingDate', 'sensor', 'groupID', 'pgeVersion', 'fileName', 'beamModeType', 's3Urls', 'temporalBaseline', 'perpendicularBaseline', 'geometry']


,sceneName,startTime,stopTime,temporalBaseline,perpendicularBaseline,geometry
0,S1A_IW_SLC__1SDV_20250415T053537_20250415T0536...,2025-04-15 05:35:37,2025-04-15T05:36:04Z,0,0,"{'coordinates': [[[10.252147, 46.595284], [6.9..."
1,S1A_IW_SLC__1SDV_20250427T053538_20250427T0536...,2025-04-27 05:35:38,2025-04-27T05:36:05Z,12,303,"{'coordinates': [[[10.248555, 46.595936], [6.9..."
2,S1C_IW_SLC__1SDV_20250503T053434_20250503T0535...,2025-05-03 05:34:34,2025-05-03T05:35:01Z,18,-344,"{'coordinates': [[[10.086966, 45.932964], [6.7..."
3,S1A_IW_SLC__1SDV_20250509T053537_20250509T0536...,2025-05-09 05:35:37,2025-05-09T05:36:04Z,24,193,"{'coordinates': [[[10.249989, 46.59584], [6.91..."
4,S1C_IW_SLC__1SDV_20250515T053435_20250515T0535...,2025-05-15 05:34:35,2025-05-15T05:35:02Z,30,-302,"{'coordinates': [[[10.086464, 45.933083], [6.7..."


In [43]:
print("Number of SAR scenes in stack_season:", len(stack_season))
print(stack_season.head())  # default is 5 rows
print(stack_season[['sceneName', 'startTime', 'temporalBaseline']].head(10))
print(stack_season.describe(include='all'))
print(stack_season.info())


Number of SAR scenes in stack_season: 10
   centerLat  centerLon              stopTime  \
0    45.9979     8.3829  2025-04-15T05:36:04Z   
1    45.9985     8.3792  2025-04-27T05:36:05Z   
2    45.3364     8.2328  2025-05-03T05:35:01Z   
3    45.9984     8.3807  2025-05-09T05:36:04Z   
4    45.3365     8.2323  2025-05-15T05:35:02Z   

                                              fileID flightDirection  \
0  S1A_IW_SLC__1SDV_20250415T053537_20250415T0536...      DESCENDING   
1  S1A_IW_SLC__1SDV_20250427T053538_20250427T0536...      DESCENDING   
2  S1C_IW_SLC__1SDV_20250503T053434_20250503T0535...      DESCENDING   
3  S1A_IW_SLC__1SDV_20250509T053537_20250509T0536...      DESCENDING   
4  S1C_IW_SLC__1SDV_20250515T053435_20250515T0535...      DESCENDING   

   pathNumber processingLevel  \
0          66             SLC   
1          66             SLC   
2          66             SLC   
3          66             SLC   
4          66             SLC   

                                

Construct SBAS interferometric pairs - seasonal + interannual 

In [42]:
sbas_pairs = set()

print('The seasonal interferometric pairs are (reference-secondary): ')
for reference, rt in stack_season.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
    secondaries = stack_season.loc[
        (stack_season.sceneName != reference)
        & (stack_season.temporalBaseline - rt <= max_temporal_baseline)
        & (stack_season.temporalBaseline - rt > 0)
    ]
    for secondary in secondaries.sceneName:
        sbas_pairs.add((reference, secondary))
        print(reference[17:25], secondary[17:25])

print('The inter-annual interferometric pairs are (reference-secondary): ')
for reference, rt in stack_season.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
    secondaries = stack_season.loc[
        (stack_season.sceneName != reference)
        & (stack_season.temporalBaseline - rt <= interannual_baseline_max)
        & (stack_season.temporalBaseline - rt > interannual_baseline_min)
    ]
    for secondary in secondaries.sceneName:
        sbas_pairs.add((reference, secondary))
        print(reference[17:25], secondary[17:25])

print('The total number of interferometric pairs are: ', len(sbas_pairs))


The seasonal interferometric pairs are (reference-secondary): 
20250608 20250614
20250602 20250608
20250602 20250614
20250527 20250602
20250527 20250608
20250527 20250614
20250521 20250527
20250521 20250602
20250521 20250608
20250521 20250614
20250515 20250521
20250515 20250527
20250515 20250602
20250515 20250608
20250515 20250614
20250509 20250515
20250509 20250521
20250509 20250527
20250509 20250602
20250509 20250608
20250509 20250614
20250503 20250509
20250503 20250515
20250503 20250521
20250503 20250527
20250503 20250602
20250503 20250608
20250427 20250503
20250427 20250509
20250427 20250515
20250427 20250521
20250427 20250527
20250427 20250602
20250415 20250427
20250415 20250503
20250415 20250509
20250415 20250515
20250415 20250521
The inter-annual interferometric pairs are (reference-secondary): 
The total number of interferometric pairs are:  38


In [44]:
# Convert interferometric pairs set to DataFrame
ifg_pairs_df = pd.DataFrame(list(sbas_pairs), columns=["reference", "secondary"])
print(ifg_pairs_df.head())


                                           reference  \
0  S1A_IW_SLC__1SDV_20250521T053537_20250521T0536...   
1  S1C_IW_SLC__1SDV_20250503T053434_20250503T0535...   
2  S1A_IW_SLC__1SDV_20250415T053537_20250415T0536...   
3  S1C_IW_SLC__1SDV_20250503T053434_20250503T0535...   
4  S1C_IW_SLC__1SDV_20250527T053437_20250527T0535...   

                                           secondary  
0  S1C_IW_SLC__1SDV_20250608T053437_20250608T0535...  
1  S1A_IW_SLC__1SDV_20250521T053537_20250521T0536...  
2  S1A_IW_SLC__1SDV_20250509T053537_20250509T0536...  
3  S1C_IW_SLC__1SDV_20250515T053435_20250515T0535...  
4  S1A_IW_SLC__1SDV_20250602T053537_20250602T0536...  


In [45]:
## Modify the path 
ifg_pairs_df.to_csv(os.path.join(work_dir,"interferometric_pairs.csv"), index=False)


Downloading the stacks in stack_season 

In [ ]:
import os
import requests
from tqdm import tqdm

# Create output folder
os.makedirs(download_dir, exist_ok=True)

# Download loop
for _, row in stack_season.iterrows():
    url = row["url"]
    filename = os.path.join(download_dir, url.split("/")[-1])

    if not os.path.exists(filename):
        print(f"Downloading {filename} ...")

        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            total_size = int(r.headers.get("Content-Length", 0))
            chunk_size = 1024 * 1024  # 1 MB

            with open(filename, 'wb') as f, tqdm(
                desc=os.path.basename(filename),
                total=total_size,
                unit='B',
                unit_scale=True,
                unit_divisor=1024
            ) as bar:
                for chunk in r.iter_content(chunk_size=chunk_size):
                    f.write(chunk)
                    bar.update(len(chunk))
    else:
        print(f"Already downloaded: {filename}")


Some summaries of metadata

In [52]:
search_df_summary = (
    search_df
    .groupby(['pathNumber', 'flightDirection'])
    .size()
    .reset_index(name='scene_count')
    .sort_values(by='scene_count', ascending=False)
)

print(search_df_summary)


   pathNumber flightDirection  scene_count
0          66      DESCENDING           14
1         139      DESCENDING           10


In [54]:
stack_season_summary = (
    stack_season
    .groupby(['pathNumber', 'flightDirection'])
    .size()
    .reset_index(name='scene_count')
    .sort_values(by='scene_count', ascending=False)
)

print("Stack Season Summary:")
print(stack_season_summary)


Stack Season Summary:
   pathNumber flightDirection  scene_count
0          66      DESCENDING           10


In [55]:
# Convert to DataFrame
sbas_df = pd.DataFrame(list(sbas_pairs), columns=["reference", "secondary"])

# Preview
print(sbas_df.head())


                                           reference  \
0  S1A_IW_SLC__1SDV_20250521T053537_20250521T0536...   
1  S1C_IW_SLC__1SDV_20250503T053434_20250503T0535...   
2  S1A_IW_SLC__1SDV_20250415T053537_20250415T0536...   
3  S1C_IW_SLC__1SDV_20250503T053434_20250503T0535...   
4  S1C_IW_SLC__1SDV_20250527T053437_20250527T0535...   

                                           secondary  
0  S1C_IW_SLC__1SDV_20250608T053437_20250608T0535...  
1  S1A_IW_SLC__1SDV_20250521T053537_20250521T0536...  
2  S1A_IW_SLC__1SDV_20250509T053537_20250509T0536...  
3  S1C_IW_SLC__1SDV_20250515T053435_20250515T0535...  
4  S1A_IW_SLC__1SDV_20250602T053537_20250602T0536...  


In [56]:
for i, pair in enumerate(sbas_pairs):
    print(f"{i+1}: {pair}")
    if i > 10:  # Stop after 10 to avoid long output
        print("... (truncated)")
        break

# Check number of elements
print(f"Number of pairs: {len(sbas_pairs)}")

# Print one or two elements to see the structure
for pair in list(sbas_pairs)[:3]:
    print(f"Type: {type(pair)}, Length: {len(pair)}, Content: {pair}")


print(f"Type of sbas_pairs: {type(sbas_pairs)}")
first = next(iter(sbas_pairs))
print(f"Type of first element: {type(first)}")
print(f"Length of first element: {len(first)}")
print(f"Contents of first element: {first}")

lengths = set(len(pair) for pair in sbas_pairs)
print(f"Unique tuple lengths in sbas_pairs: {lengths}")


1: ('S1A_IW_SLC__1SDV_20250521T053537_20250521T053604_059288_075B98_4642', 'S1C_IW_SLC__1SDV_20250608T053437_20250608T053504_002687_0058C3_BB3E')
2: ('S1C_IW_SLC__1SDV_20250503T053434_20250503T053501_002162_004973_6140', 'S1A_IW_SLC__1SDV_20250521T053537_20250521T053604_059288_075B98_4642')
3: ('S1A_IW_SLC__1SDV_20250415T053537_20250415T053604_058763_074791_1397', 'S1A_IW_SLC__1SDV_20250509T053537_20250509T053604_059113_07558F_8B12')
4: ('S1C_IW_SLC__1SDV_20250503T053434_20250503T053501_002162_004973_6140', 'S1C_IW_SLC__1SDV_20250515T053435_20250515T053502_002337_004EE6_7427')
5: ('S1C_IW_SLC__1SDV_20250527T053437_20250527T053504_002512_0053BB_AD31', 'S1A_IW_SLC__1SDV_20250602T053537_20250602T053604_059463_0761AE_4611')
6: ('S1C_IW_SLC__1SDV_20250503T053434_20250503T053501_002162_004973_6140', 'S1A_IW_SLC__1SDV_20250602T053537_20250602T053604_059463_0761AE_4611')
7: ('S1A_IW_SLC__1SDV_20250509T053537_20250509T053604_059113_07558F_8B12', 'S1C_IW_SLC__1SDV_20250608T053437_20250608T053504

Functions to download the DEM 

In [58]:
def print_aster_wget_instructions(lat_tile='N46', lon_tile='E008'):
    """
    Print instructions and the exact wget command for downloading an ASTER GDEM tile.
    
    Args:
        lat_tile (str): Latitude tile name, e.g., 'N46'
        lon_tile (str): Longitude tile name, e.g., 'E008'
    """
    date_path = "2020.01.01"
    base_url = f"https://e4ftl01.cr.usgs.gov/ASTT/ASTGTM.003/{date_path}"
    tile_name = f"ASTGTMV003_{lat_tile}{lon_tile}_dem.tif"

    print("ASTER GDEM Download Instructions")
    print("1. Make sure you have a ~/.netrc file with your NASA Earthdata login credentials:")
    print("""
machine urs.earthdata.nasa.gov
login your_earthdata_username
password your_earthdata_password
""")
    print("2. Then run the following command in your shell:\n")
    print(f"""wget --load-cookies ~/.urs_cookies \\
     --save-cookies ~/.urs_cookies \\
     --keep-session-cookies \\
     --auth-no-challenge=on \\
     --user-agent="Mozilla/5.0" \\
     --netrc -c {base_url}/{tile_name}\n""")
    print("Tile:", tile_name)
    print("Output: Downloaded .tif file for the specified ASTER tile.")

# Example usage
print_aster_wget_instructions('N46', 'E008')


ASTER GDEM Download Instructions
1. Make sure you have a ~/.netrc file with your NASA Earthdata login credentials:

machine urs.earthdata.nasa.gov
login your_earthdata_username
password your_earthdata_password

2. Then run the following command in your shell:

wget --load-cookies ~/.urs_cookies \
     --save-cookies ~/.urs_cookies \
     --keep-session-cookies \
     --auth-no-challenge=on \
     --user-agent="Mozilla/5.0" \
     --netrc -c https://e4ftl01.cr.usgs.gov/ASTT/ASTGTM.003/2020.01.01/ASTGTMV003_N46E008_dem.tif

Tile: ASTGTMV003_N46E008_dem.tif
Output: Downloaded .tif file for the specified ASTER tile.


In [68]:
import subprocess
from tqdm import tqdm
import os

def download_dem_with_make_dem(W, E, S, N, download_dir, mode=1):
    """
    Download DEM using GMTSAR's make_dem.csh script.

    Parameters:
    - W, E: Western and Eastern longitudes
    - S, N: Southern and Northern latitudes
    - download_dir: directory to save the DEM
    - mode: resolution mode (1 = 1s, 2 = 3s), default is 1
    """
    print(f"[INFO] Downloading DEM for W:{W} E:{E} S:{S} N:{N} with mode {mode}")
    print(f"[INFO] Output directory: {download_dir}")
    
    os.makedirs(download_dir, exist_ok=True)

    cmd = ["make_dem.csh", str(W), str(E), str(S), str(N), str(mode)]
    
    with tqdm(total=1, desc="DEM Download (make_dem.csh)") as pbar:
        subprocess.run(cmd, cwd=download_dir, check=True)
        pbar.update(1)


In [61]:
import os
from pathlib import Path
def download_earth_relief_dem(minlat, maxlat, minlon, maxlon, resolution='03s', download_dir='raw/dem/'):
    """
    Download DEM using GMT's @earth_relief.

    Args:
        minlat, maxlat, minlon, maxlon: float, bounding box
        resolution: str, e.g. '03s', '15s', etc.
        download_dir: str, output directory (default: 'raw/dem/')
    """
    
    Path(download_dir).mkdir(parents=True, exist_ok=True)
    print("GMT earth_relief DEM download:")
    print(f"cd {download_dir}")
    print(f"gmt grdcut @earth_relief_{resolution} -R{minlon}/{maxlon}/{minlat}/{maxlat} -Gdem_relief.grd")

    # Optional: auto-download with subprocess
    # subprocess.run(['gmt', 'grdcut', f'@earth_relief_{resolution}',
    #                 f'-R{minlon}/{maxlon}/{minlat}/{maxlat}', '-Gdem_relief.grd'],
    #                cwd=download_dir)


In [ ]:
# This one uses ISCE2 dem.py 
import subprocess

cmd = [
    "python", "/path/to/isce2/applications/dem.py",
    "-a", "SRTM1",
    "-b", "44.1 47.2 6.2 10.5",
    "-r", "1",
    "-f", "dem_wide"
]

subprocess.run(cmd, check=True)


In [107]:
# === Download DEM using GMTSAR's make_dem.csh ===
#from tqdm.notebook import tqdm
from tqdm import tqdm
import subprocess

print("\n Downloading DEM using make_dem.csh...")


# for _ in tqdm(range(1), desc="DEM Download"):
#     download_dem_with_make_dem(
#         W=bbox[0],
#         E=bbox[2],
#         S=bbox[1],
#         N=bbox[3],
#         download_dir=dem_dir,
#         mode=1
#     )

for _ in tqdm(range(1), desc="DEM Download"):
    download_dem_with_make_dem(
        6,
        12,
        39,
        50,
        download_dir=dem_dir,
        mode=2
    )



DEM Download:   0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Downloading DEM for W:6 E:12 S:39 N:50 with mode 2
[INFO] Output directory: /mnt/data/gmtsar_test_1/topo



START: make_dem.csh



grdblend [NOTICE]: Remote data courtesy of GMT data server oceania [http://oceania.generic-mapping-tools.org]

grdblend [NOTICE]: Earth Relief at 3x3 arc seconds tiles provided by SRTMGL3 (land only) [NASA/USGS].
grdblend [NOTICE]:   -> Download 1x1 degree grid tile (earth_relief_03s_g): N39E008
grdblend [NOTICE]:   -> Download 1x1 degree grid tile (earth_relief_03s_g): N39E009
grdblend [NOTICE]:   -> Download 1x1 degree grid tile (earth_relief_03s_g): N40E008
grdblend [NOTICE]:   -> Download 1x1 degree grid tile (earth_relief_03s_g): N40E009
grdblend [NOTICE]:   -> Download 1x1 degree grid tile (earth_relief_03s_g): N41E008
grdblend [NOTICE]:   -> Download 1x1 degree grid tile (earth_relief_03s_g): N41E009
grdblend [NOTICE]:   -> Download 1x1 degree grid tile (earth_relief_03s_g): N41E011
grdblend [NOTICE]:   -> Download 1x1 degree grid tile (earth_relief_03s_g): N42E006
grdblend [NOTICE]:   -> Download 1x1 degree grid tile (earth_relief_03s_g): N42E008
grdblend [NOTICE]:   -> Downloa


created dem.grd, heights relative to WGS84 ellipsoid

END: make_dem.csh



Downloading precise orbit using sentineleof 0.11.0

In [ ]:
##Need to instal: pip install --upgrade sentineleof ver 0.11.0 for S1C

# === Download orbits for all SLCs under /raw ===

print("\n Searching for SLC path folders to fetch orbits...")
#from tqdm.notebook import tqdm
from tqdm import tqdm
paths = glob.glob(os.path.join(download_dir, "path_*"))

for download_dir in tqdm(paths, desc="Orbit Download", unit="path"):
    tqdm.write(f"Downloading orbits for: {download_dir}")
    subprocess.run([
        "eof",
        "--search-path", download_dir,
        "--save-dir", orbit_dir,
        "--force-asf"
    ], check=True)


Select a pair before and after an incident 

In [70]:
incident_date = pd.to_datetime("2025-05-28")

before_scene = stack_season[stack_season['startTime'] < incident_date].sort_values('startTime').iloc[-1]
after_scene = stack_season[stack_season['startTime'] > incident_date].sort_values('startTime').iloc[0]

print("Reference (before landslide):", before_scene['sceneName'], before_scene['startTime'])
print("Secondary (after landslide):", after_scene['sceneName'], after_scene['startTime'])


Reference (before landslide): S1C_IW_SLC__1SDV_20250527T053437_20250527T053504_002512_0053BB_AD31 2025-05-27 05:34:37
Secondary (after landslide): S1A_IW_SLC__1SDV_20250602T053537_20250602T053604_059463_0761AE_4611 2025-06-02 05:35:37


In [75]:
before_scene_A = stack_season[(stack_season['platform'] == 'Sentinel-1A') & 
                                (stack_season['startTime'] < incident_date)].sort_values('startTime').iloc[-1]
after_scene_A  = stack_season[(stack_season['platform'] == 'Sentinel-1A') & 
                                (stack_season['startTime'] > incident_date)].sort_values('startTime').iloc[0]

print("Reference (before landslide) S1A:", before_scene_A['sceneName'], before_scene_A['startTime'])
print("Secondary (after landslide) S1A:", after_scene_A['sceneName'], after_scene_A['startTime'])


Reference (before landslide) S1A: S1A_IW_SLC__1SDV_20250521T053537_20250521T053604_059288_075B98_4642 2025-05-21 05:35:37
Secondary (after landslide) S1A: S1A_IW_SLC__1SDV_20250602T053537_20250602T053604_059463_0761AE_4611 2025-06-02 05:35:37


In [77]:
before_scene_C = stack_season[(stack_season['platform'] == 'Sentinel-1C') & 
                                (stack_season['startTime'] < incident_date)].sort_values('startTime').iloc[-1]
after_scene_C  = stack_season[(stack_season['platform'] == 'Sentinel-1C') & 
                                (stack_season['startTime'] > incident_date)].sort_values('startTime').iloc[0]

print("Reference (before landslide) S1C:", before_scene_C['sceneName'], before_scene_C['startTime'])
print("Secondary (after landslide) S1C:", after_scene_C['sceneName'], after_scene_C['startTime'])


Reference (before landslide) S1C: S1C_IW_SLC__1SDV_20250527T053437_20250527T053504_002512_0053BB_AD31 2025-05-27 05:34:37
Secondary (after landslide) S1C: S1C_IW_SLC__1SDV_20250608T053437_20250608T053504_002687_0058C3_BB3E 2025-06-08 05:34:37


In [87]:
import os
from pathlib import Path

# P2P processing directory
p2p_dir = os.path.join(work_dir, "p2p")
os.makedirs(p2p_dir, exist_ok=True)

# Use the actual zip file names from your search results
scene_names = [before_scene['sceneName'], after_scene['sceneName']]

# Link .zip files (SAFE archives)
for scene_name in scene_names:
    zip_file = scene_name + ".zip"
    src = os.path.join(download_dir, zip_file)
    dst = os.path.join(p2p_dir, zip_file)
    if not os.path.exists(dst):
        os.symlink(src, dst)

# Link orbit .EOF files as before
for file in os.listdir(download_dir):
    if file.endswith(".EOF"):
        src = os.path.join(download_dir, file)
        dst = os.path.join(p2p_dir, file)
        if not os.path.exists(dst):
            os.symlink(src, dst)


In [79]:
pair_name = f"p2p_{before_scene['platform']}_{after_scene['platform']}"
p2p_dir = os.path.join(work_dir, pair_name)
os.makedirs(p2p_dir, exist_ok=True)


In [88]:
p2p_txt = os.path.join(p2p_dir, "p2p.txt")
with open(p2p_txt, "w") as f:
    f.write(before_scene['fileName'] + "\n")
    f.write(after_scene['fileName'] + "\n")
print(f"Created: {p2p_txt}")
print(f"Created p2p.txt with:\n- {before_scene['fileName']}\n- {after_scene['fileName']}")


Created: /mnt/data/gmtsar_test_1/p2p/p2p.txt
Created p2p.txt with:
- S1C_IW_SLC__1SDV_20250527T053437_20250527T053504_002512_0053BB_AD31.zip
- S1A_IW_SLC__1SDV_20250602T053537_20250602T053604_059463_0761AE_4611.zip


In [93]:
# ✅ Cell 1: Stitch the SAFE folders using prep_raw_S1_TOPS
from pathlib import Path
import subprocess

# Define the master and slave scene .SAFE folders
master_safe = next(Path(download_dir).glob("S1C_IW_SLC__1SDV_20250527T053437_*.SAFE"))
slave_safe = next(Path(download_dir).glob("S1A_IW_SLC__1SDV_20250602T053537_*.SAFE"))

# Run prep_raw_S1_TOPS.csh on both scenes
print("Running prep_raw_S1_TOPS.csh for master...")
subprocess.run(["prep_raw_S1_TOPS.csh", str(master_safe)], check=True)

print("Running prep_raw_S1_TOPS.csh for slave...")
subprocess.run(["prep_raw_S1_TOPS.csh", str(slave_safe)], check=True)

print("✅ SAFE scenes stitched. Ready for p2p processing.")


Running prep_raw_S1_TOPS.csh for master...


FileNotFoundError: [Errno 2] No such file or directory: 'prep_raw_S1_TOPS.csh'

In [94]:
reference = before_scene['sceneName'].replace(".zip", "")
secondary = after_scene['sceneName'].replace(".zip", "")

print(f"🛰️ Running p2p_S1_TOPS_Frame.csh with:")
print(f"Master  : {reference}")
print(f"Secondary: {secondary}")

subprocess.run(["p2p_S1_TOPS_Frame.csh", reference, secondary], cwd=work_dir, check=True)


🛰️ Running p2p_S1_TOPS_Frame.csh with:
Master  : S1C_IW_SLC__1SDV_20250527T053437_20250527T053504_002512_0053BB_AD31
Secondary: S1A_IW_SLC__1SDV_20250602T053537_20250602T053604_059463_0761AE_4611

Usage: p2p_S1_TOPS_Frame.csh Master.SAFE Master.EOF Aligned.SAFE Aligned.EOF config.s1a.txt polarization parallel

Example: p2p_S1_TOPS_Frame.csh S1A_IW_SLC__1SDV_20150607T014936_20150607T015003_006261_00832E_3626.SAFE S1A_OPER_AUX_POEORB_OPOD_20150615T155109_V20150525T225944_20150527T005944.EOF S1A_IW_SLC__1SSV_20150526T014935_20150526T015002_006086_007E23_679A.SAFE S1A_OPER_AUX_POEORB_OPOD_20150627T155155_V20150606T225944_20150608T005944.EOF config.s1a.txt vv 1

    Place the .SAFE file in the raw folder, DEM in the topo folder
    During processing, F1, F2, F3 and merge folder will be generated
    Final results will be placed in the merge folder, with phase
    corr [unwrapped phase].
    polarization = vv vh hh or hv 
    parallel = 0-sequential  1-parallel 

Reference: Xu, X., Sandwell,

CalledProcessError: Command '['p2p_S1_TOPS_Frame.csh', 'S1C_IW_SLC__1SDV_20250527T053437_20250527T053504_002512_0053BB_AD31', 'S1A_IW_SLC__1SDV_20250602T053537_20250602T053604_059463_0761AE_4611']' returned non-zero exit status 1.

In [ ]:


# ✅ Cell 2: Run p2p_processing.csh
from pathlib import Path
import subprocess

# Extract stems (filenames without extension) from stitched products
master_stem = Path(master_safe).stem
slave_stem = Path(slave_safe).stem

print("Running p2p_processing.csh ...")
subprocess.run([
    "p2p_processing.csh",
    "S1_TOPS",
    master_stem,
    slave_stem
], cwd=work_dir, check=True)

print("✅ GMTSAR p2p processing completed.")


In [91]:
import zipfile
from pathlib import Path

# Define SAFE filenames
master_zip = f"{before_scene['sceneName']}.zip"
slave_zip = f"{after_scene['sceneName']}.zip"

# Paths
master_path = Path(download_dir) / master_zip
slave_path = Path(download_dir) / slave_zip

# Unzip if needed
for zip_path in [master_path, slave_path]:
    target_dir = zip_path.with_suffix(".SAFE")
    if not target_dir.exists():
        print(f"🔓 Extracting {zip_path.name} ...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(download_dir)
    else:
        print(f"✅ Already unzipped: {target_dir.name}")


🔓 Extracting S1C_IW_SLC__1SDV_20250527T053437_20250527T053504_002512_0053BB_AD31.zip ...
🔓 Extracting S1A_IW_SLC__1SDV_20250602T053537_20250602T053604_059463_0761AE_4611.zip ...


In [92]:
import subprocess

# SAT type for Sentinel-1 IW SLCs
sat = "S1_TOPS"

# Strip .zip from filenames
master = before_scene['fileName'].replace(".zip", "")
slave = after_scene['fileName'].replace(".zip", "")

print(f"Master (reference): {master}")
print(f"Slave (secondary): {slave}")

cmd = ["p2p_processing.csh", sat, master, slave]

try:
    subprocess.run(cmd, cwd=work_dir, check=True)
    print("GMTSAR p2p processing completed.")
except subprocess.CalledProcessError as e:
    print(f"Processing failed: {e}")
    

Master (reference): S1C_IW_SLC__1SDV_20250527T053437_20250527T053504_002512_0053BB_AD31
Slave (secondary): S1A_IW_SLC__1SDV_20250602T053537_20250602T053604_059463_0761AE_4611


PREPROCESS - START

Working on images S1C_IW_SLC__1SDV_20250527T053437_20250527T053504_002512_0053BB_AD31 S1A_IW_SLC__1SDV_20250602T053537_20250602T053604_059463_0761AE_4611 ...
 no file  raw/S1C_IW_SLC__1SDV_20250527T053437_20250527T053504_002512_0053BB_AD31.xml
GMTSAR p2p processing completed.


##NOW TESTING AGAIN IN A NEW DIRECTORY BASED ON https://gmtsar.github.io/documentation/S1_Batch_Preprocessing.html

In [95]:
from pathlib import Path

# Define root
root_data = Path("/mnt/data")
project_dir = root_data / "gmtsar_test_2"

# Define top-level directories
top_dirs = ["data", "orbit", "reframed", "topo", "F1", "F2", "F3", "SBAS", "merge"]

# Define F* subdirectories
f_subdirs = ["raw", "SLC", "intf_in", "topo"]

# Create top-level dirs
for d in top_dirs:
    (project_dir / d).mkdir(parents=True, exist_ok=True)

# Create F1/F2/F3 substructure
for f in ["F1", "F2", "F3"]:
    for sub in f_subdirs:
        (project_dir / f / sub).mkdir(parents=True, exist_ok=True)

# Touch the batch_tops.config file inside each F*
for f in ["F1", "F2", "F3"]:
    config_file = project_dir / f / "batch_tops.config"
    config_file.touch()

print(f"Directory structure created under: {project_dir}")


Directory structure created under: /mnt/data/gmtsar_test_2


In [96]:
!tree -d -L 3 /mnt/data/gmtsar_test_2


/mnt/data/gmtsar_test_2
├── F1
│   ├── SLC
│   ├── intf_in
│   ├── raw
│   └── topo
├── F2
│   ├── SLC
│   ├── intf_in
│   ├── raw
│   └── topo
├── F3
│   ├── SLC
│   ├── intf_in
│   ├── raw
│   └── topo
├── SBAS
├── data
├── merge
├── orbit
├── reframed
└── topo

21 directories


Get the satellite footpring from LED file 

In [103]:
import subprocess
from pyproj import Transformer
import numpy as np
import os

# Set your raw directory path (adjust if needed)
raw_dir = "/mnt/data/gmtsar_test_1/F1/raw"
led_files = [f for f in os.listdir(raw_dir) if f.endswith("F1.LED")]

# Transformer from ECEF to lat/lon
transformer = Transformer.from_crs("epsg:4978", "epsg:4326", always_xy=True)

swath_bounds = {}

for led_file in led_files:
    swath = led_file.split("_")[2]  # E.g., 053438
    path = os.path.join(raw_dir, led_file)

    # Extract first and last line ECEF coordinates
    awk_cmd = f"awk 'NR==2{{print $4, $5, $6}} END{{print $4, $5, $6}}' {path}"
    result = subprocess.check_output(awk_cmd, shell=True).decode().strip().split("\n")
    xyz_start = list(map(float, result[0].split()))
    xyz_end = list(map(float, result[1].split()))
    print(f"[Swath {swath}] Start ECEF: {xyz_start}, End ECEF: {xyz_end}")
    # Convert to lat/lon
    lon1, lat1, _ = transformer.transform(xyz_start[0], xyz_start[1], xyz_start[2])
    lon2, lat2, _ = transformer.transform(xyz_end[0], xyz_end[1], xyz_end[2])

    # Get bounding box
    W = round(min(lon1, lon2), 4)
    E = round(max(lon1, lon2), 4)
    S = round(min(lat1, lat2), 4)
    N = round(max(lat1, lat2), 4)

    swath_bounds[swath] = {"W": W, "E": E, "S": S, "N": N}
    print(f"[Swath {swath}] W: {W}, E: {E}, S: {S}, N: {N}")

# === Merge bounding boxes ===
W_all = min(b["W"] for b in swath_bounds.values())
E_all = max(b["E"] for b in swath_bounds.values())
S_all = min(b["S"] for b in swath_bounds.values())
N_all = max(b["N"] for b in swath_bounds.values())

print("\n[Combined Coverage for All Swaths]")
print(f"W: {W_all}, E: {E_all}, S: {S_all}, N: {N_all}")

# === Check DEM coverage using GMT ===
dem_path = "/mnt/data/gmtsar_test_1/F1/topo/dem.grd"
try:
    grdinfo = subprocess.check_output(f"gmt grdinfo {dem_path}", shell=True).decode()
    print("\n[DEM grdinfo Output]")
    print(grdinfo)
except subprocess.CalledProcessError:
    print("[ERROR] Could not read dem.grd — check path or GMT install.")


[Swath ALL] Start ECEF: [-4654596.487311, -223083.269757, 5315068.231733], End ECEF: [5351057.468749, -620367.425822, -4596147.382489]
[Swath ALL] W: -177.2561, E: -6.613, S: -40.6421, N: 48.9294
[Swath ALL] Start ECEF: [-4714040.444577, -247922.081594, 5261442.231315], End ECEF: [5405304.917568, -606441.635012, -4534014.418914]
[Swath ALL] W: -176.9895, E: -6.4015, S: -39.9842, N: 48.2738
[Swath 053438] Start ECEF: [-4654596.487311, -223083.269757, 5315068.231733], End ECEF: [5351057.468749, -620367.425822, -4596147.382489]
[Swath 053438] W: -177.2561, E: -6.613, S: -40.6421, N: 48.9294
[Swath 053538] Start ECEF: [-4714040.444577, -247922.081594, 5261442.231315], End ECEF: [5405304.917568, -606441.635012, -4534014.418914]
[Swath 053538] W: -176.9895, E: -6.4015, S: -39.9842, N: 48.2738

[Combined Coverage for All Swaths]
W: -177.2561, E: -6.4015, S: -40.6421, N: 48.9294

[DEM grdinfo Output]
/mnt/data/gmtsar_test_1/F1/topo/dem.grd: Title: Produced by grdmath
/mnt/data/gmtsar_test_1/F1

In [74]:
#print("Available columns:\n", stack_season.columns.tolist())
print(stack_season[['sceneName', 'platform', 'temporalBaseline']].head(3))

                                           sceneName     platform  \
0  S1A_IW_SLC__1SDV_20250415T053537_20250415T0536...  Sentinel-1A   
1  S1A_IW_SLC__1SDV_20250427T053538_20250427T0536...  Sentinel-1A   
2  S1C_IW_SLC__1SDV_20250503T053434_20250503T0535...  Sentinel-1C   

   temporalBaseline  
0                 0  
1                12  
2                18  
